## Dynamic Routing

This implementaiton solves a scenario using the dynamic routing.

- If user enters the correct answer to quesiton given a more difficult question
- If the user gives wrong answer then the user is given an explanation of answer.

<p align="center">
<img width="400" alt="QA_user_problem_flow"
src="../images/dynamic_routing_langchian/QA_user_problem_flow.png" />
</p>

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain_core
!pip install langchain_community

!pip install langchain-google-genai
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate


classifier_template = PromptTemplate(
    input_variables = ["question", "answer"],
    template= """
    You are given a question and the usser's response to the question. Classify the response as either `Correct` or `Incorrect`.
    Do not respond with more than one word.
    Question - {question}
    User's Answer - {answer}
    Classification:
    """
)

llm = ChatOpenAI()

output_parser = StrOutputParser()

classifier_chain = classifier_template | llm | output_parser

classifier_chain.invoke({"question":"what are penguins", "answer":"Penguins are sea animals"})

'Correct'

In [ ]:
correct_template = PromptTemplate(
    input_variables = ["question"],
    template= """
    The user asked the following question and answered it correctly. Now ask a more difficult question from the user on same topic.
    Question - {question}
    New Question:
    """
)

correct_chain = correct_template | llm | output_parser

incorrect_template = PromptTemplate(
    input_variables = ["question"],
    template= """
    The user asked the following question and answered it incorrectly. Give the correct answer and explain to the user.
    Question - {question}
    Correct Answer:
    Explanation:
    """
)

incorrect_chain = incorrect_template | llm | output_parser

In [ ]:
def route(info):
  if info["result"].lower() == "correct":
    return correct_chain
  elif info["result"].lower() == "incorrect":
    return incorrect_chain
  else:
    return "Format is not Correct"

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough


final_chain = (
    RunnableParallel({
        "result": classifier_chain,
        "question": lambda x: x["question"],
        "answer": lambda x: x["answer"]
    })
    |
    RunnableParallel({
        "response": RunnableLambda(route),
        "input": RunnablePassthrough()
    })
)

final_chain.invoke({"question":"what are penguins", "answer":"Penguins are birds."})

{'response': 'What is the average lifespan of a penguin in the wild?',
 'input': {'result': 'Correct',
  'question': 'what are penguins',
  'answer': 'Penguins are birds.'}}

In [ ]:
final_chain.invoke({"question":"what are penguins", "answer":"Penguins are mamals."})

{'response': 'Penguins are flightless birds that are native to the Southern Hemisphere. They are known for their black and white plumage, waddling walk, and their ability to swim underwater. Penguins primarily feed on fish and other marine life. They are well-adapted to life in cold climates and often live in large colonies called rookeries. Penguins come in different species, such as the Emperor Penguin, King Penguin, Gentoo Penguin, and Adelie Penguin.',
 'input': {'result': 'Incorrect',
  'question': 'what are penguins',
  'answer': 'Penguins are mamals.'}}